In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../data/raw/test.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Id_old   3799 non-null   int64  
 1   Id       3799 non-null   int64  
 2   GP       3799 non-null   int64  
 3   MIN      3799 non-null   float64
 4   PTS      3799 non-null   float64
 5   FGM      3799 non-null   float64
 6   FGA      3799 non-null   float64
 7   FG%      3799 non-null   float64
 8   3P Made  3799 non-null   float64
 9   3PA      3799 non-null   float64
 10  3P%      3799 non-null   float64
 11  FTM      3799 non-null   float64
 12  FTA      3799 non-null   float64
 13  FT%      3799 non-null   float64
 14  OREB     3799 non-null   float64
 15  DREB     3799 non-null   float64
 16  REB      3799 non-null   float64
 17  AST      3799 non-null   float64
 18  STL      3799 non-null   float64
 19  BLK      3799 non-null   float64
 20  TOV      3799 non-null   float64
dtypes: float64(18)

No non-null and all values are numeric

In [4]:
df.describe(include='all')

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
count,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000
mean,7010.614109,1899.000000,62.853909,18.650224,7.328034,2.835404,6.302580,44.599079,0.255962,0.796920,19.234746,1.399842,1.953567,71.612924,1.096025,2.179495,3.275783,1.636483,0.653593,0.257726,1.257910
std,3954.173641,1096.821164,17.151740,8.727259,4.294724,1.688427,3.579221,6.040168,0.380987,1.052862,15.968989,0.926140,1.250376,10.457336,0.785678,1.371935,2.070646,1.335496,0.410573,0.639660,0.712449
min,1.000000,0.000000,6.000000,3.700000,0.700000,0.300000,0.800000,25.100000,-1.000000,-2.700000,-38.000000,0.000000,0.000000,23.700000,0.000000,0.200000,0.300000,0.000000,0.000000,-7.100000,0.100000
25%,3644.000000,949.500000,51.000000,12.200000,4.200000,1.600000,3.700000,40.500000,0.000000,0.100000,8.500000,0.700000,1.000000,65.000000,0.500000,1.200000,1.800000,0.600000,0.400000,0.100000,0.700000
50%,7062.000000,1899.000000,63.000000,17.000000,6.400000,2.500000,5.500000,44.600000,0.300000,0.800000,19.400000,1.200000,1.700000,71.500000,0.900000,1.900000,2.800000,1.300000,0.600000,0.200000,1.100000
75%,10402.500000,2848.500000,74.000000,23.300000,9.400000,3.700000,8.100000,48.500000,0.500000,1.500000,30.250000,1.900000,2.600000,78.000000,1.500000,2.900000,4.300000,2.300000,0.900000,0.400000,1.600000
max,13792.000000,3798.000000,126.000000,68.000000,33.000000,13.400000,26.200000,74.600000,1.600000,4.300000,73.800000,7.800000,9.800000,127.100000,6.900000,12.000000,18.500000,9.000000,2.700000,14.800000,5.200000


## Data Quality Concerns
- GP, 3PM, 3PA, 3p%, FT%, BLK have negative value as minimum - **Convert to absolute value** - Done
- There are potential outliers in the value between 75% mark and MAX is huge jump across all features
- Check if all made value is smaller than attempt value
- All % values are bit off and not close to Made/attempt - **Consider dropping these fields or recreate them** - Done
- BLK has outliers - **Fix it or drop these**
- Use ensemble approach - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html build 25 random weak regressors and let them vote on the prediction
- Use **cross-validation** to fully utilize the whole training set
- 3PA and FTA has 0 value - Need to ensure 3PM and FTM are also 0 in these cases
- Split dataset and use different models for each set
    - GP < 30 ... Overfitting
    - GP > 30 ... Overfitting
    
- Target the concept of ROC, set threshold and convert high and low probability to 1 and 0, and leave the grey area as float - Doesn't do much
- Convert GP and MIN to be catagorical data like small, medium and large, then use one-hot encoding or 1~5 (This takes away the linear relationship and focus on what's important) - **Having all features help with ROC score**
- ROC plot to see if the problem or confusion map to further understand the problem
- Rip out outliers


## Featureunion - Same set of data being run in parallel   
- It can convert/derive original features into new features (the prediction result - multiple estimators)
- Then use these new features to run an estimator against ground truth to form ensemble
- This brings the best out of estimator collection

## Pipeline
- Define shared steps as pipeline
- Include pipeline within a pipeline for simple reference and reuse
- Then have evaluation to take model/pipeline as input for output and scoring
- Try outputting pipeline as artifacts for reuse